## Selecting the fingerprints

In this notebook I'm going to select the fingerprints that store the information in a more appropriate manner for a CNN model of 1 dimension. To do that, I'm going to program a script that uses different fingerprints to train a CNN and I will chose the ones giving a model with the best accuracy. The model is build to be fast to speed up the process.

## Imports

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPool1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
import warnings

### Load data

In [2]:
pd.options.mode.chained_assignment = None

In [20]:
drugs = pd.read_pickle(os.path.join('res', 'pickles', 'all_label_drugs.pkl')).drop('FeatInvariants', axis=1) #I discard the Feature Invariants because they give an error I have to look into yet
display(drugs.head())
drugs.iloc[0]['Morgan2FP'].shape
drugs.info()
drugs.dropna(inplace=True)

,CID,ATC_Code_Short,Molecule,ATC_Code_#,ConnInvariants,Morgan2FP,MACCSKeys,AtomPairFP,TopTorFP,AvalonFP,PubchemFP,CactvsFP
0,1,N,<rdkit.Chem.rdchem.Mol object at 0x0000020A312...,10,"[2246728737, 2246699815, 864942730, 864674487,...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, ..."
1,119,N,<rdkit.Chem.rdchem.Mol object at 0x0000020A312...,10,"[2245384272, 2245384272, 2246699815, 864942730...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, ..."
2,137,L,<rdkit.Chem.rdchem.Mol object at 0x0000020A312...,8,"[2245384272, 2245384272, 2246699815, 864942730...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, ..."
3,176,G,<rdkit.Chem.rdchem.Mol object at 0x0000020A312...,4,"[2246728737, 2246699815, 864942730, 864662311]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, ..."
4,187,S,<rdkit.Chem.rdchem.Mol object at 0x0000020A312...,14,"[2246728737, 2246699815, 864942730, 864674487,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, ..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10183 entries, 0 to 10182
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   CID             10183 non-null  int64 
 1   ATC_Code_Short  10183 non-null  object
 2   Molecule        10183 non-null  object
 3   ATC_Code_#      10183 non-null  int32 
 4   ConnInvariants  10183 non-null  object
 5   Morgan2FP       10183 non-null  object
 6   MACCSKeys       10183 non-null  object
 7   AtomPairFP      10183 non-null  object
 8   TopTorFP        10183 non-null  object
 9   AvalonFP        10183 non-null  object
 10  PubchemFP       10183 non-null  object
 11  CactvsFP        10183 non-null  object
dtypes: int32(1), int64(1), object(10)
memory usage: 915.0+ KB


## Functions

In [22]:
def select_fp(column_list):
    """
    Function that takes a list of columns and makes a train/test split for each of them with the column containing the labels as y.
    Then returns a dictionary with the column name as a key and the tuple with X_train, X_test, y_train, y_test as value.
    Input: list with column names
    Output: dictionary
    """
    splits_dic = {}
    for column in column_list:
        X = drugs[column]
        y = drugs['ATC_Code_#'] #atc_code_# drug_class_code	
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        splits_dic[column] = X_train, X_test, y_train, y_test
    return splits_dic

In [23]:
fp_list = list(drugs.columns[5:])
splits_dic = select_fp(fp_list)
print(type(splits_dic['Morgan2FP'][0]))
print(splits_dic['Morgan2FP'][0].shape)
print(splits_dic['Morgan2FP'][0])

<class 'pandas.core.series.Series'>
(7128,)
8947    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5371    [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3397    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...
3082    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5474    [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...
                              ...                        
5734    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5191    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5390    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
860     [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
7270    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: Morgan2FP, Length: 7128, dtype: object


In [24]:
def build_model(inshape, nclasses):
    """
    Function that builds a CNN model with non optimal conditions.
    Input: tuple with inner shape of arrays, integer with number of classes
    Output: a CNN model
    """
    model = Sequential()
    model.add(Conv1D(100, 9, activation='relu', kernel_initializer='he_uniform', input_shape=inshape))
    model.add(MaxPool1D(2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.5))
    model.add(Dense(nclasses, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

In [25]:
def reshape_and_build(splits_dic):
    """
    Function that takes the dictionary generated by "select_fp" and reshapes X_train to fit a CNN model.
    Then builds the CNN model using the function 'build_model'. Return both the reshaped arrays and the models.
    Input: dictionary with column names as keys and a tuple of arrays as values
    Output: dictionary with column names as keys and a tuple with both reshaped arrays and CNN models 
    """
    arrays_models_dic = {}
    for column, tup in splits_dic.items():
        print(column)
        X_train = np.array(list(tup[0]))
        print('Shape X_train: ', X_train.shape, type(X_train))
        display(X_train[0])
        X_test = np.array(list(tup[1]))
        n_classes = len(np.unique(tup[2]))
        print('Number of classes: ', n_classes)
        X_train= X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_test= X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
        print('Reshaped X_train: ', X_train.shape)
        in_shape = X_train.shape[1:]
        print('In_shape: ', in_shape)
        print('Shape y_train: ', tup[2].shape)
        
        model = build_model(in_shape, n_classes)
        arrays_models_dic[column] = X_train, X_test, tup[2], tup[3], model

    return arrays_models_dic

In [26]:
array_model_dic= reshape_and_build(splits_dic)

Morgan2FP
Shape X_train:  (7128, 2048) <class 'numpy.ndarray'>


array([0, 0, 0, ..., 0, 0, 0])

Number of classes:  16
Reshaped X_train:  (7128, 2048, 1)
In_shape:  (2048, 1)
Shape y_train:  (7128,)
MACCSKeys
Shape X_train:  (7128, 167) <class 'numpy.ndarray'>


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0])

Number of classes:  16
Reshaped X_train:  (7128, 167, 1)
In_shape:  (167, 1)
Shape y_train:  (7128,)
AtomPairFP
Shape X_train:  (7128, 2048) <class 'numpy.ndarray'>


...

Number of classes:  16
Reshaped X_train:  (7128, 2048, 1)
In_shape:  (2048, 1)
Shape y_train:  (7128,)
PubchemFP
Shape X_train:  (7128, 898) <class 'numpy.ndarray'>


array([1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,

Number of classes:  16
Reshaped X_train:  (7128, 898, 1)
In_shape:  (898, 1)
Shape y_train:  (7128,)
CactvsFP
Shape X_train:  (7128, 898) <class 'numpy.ndarray'>


array([1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,

Number of classes:  16
Reshaped X_train:  (7128, 898, 1)
In_shape:  (898, 1)
Shape y_train:  (7128,)


In [27]:
def fit_model(arrays_models_dic):
    """
    Function that takes the dictionary generated by 'reshape_and_build' and fits the CNN model contained in the values with the X_train and y_train arrays also in the values.
    Then evaluates the model against X_test e y_test and returns the accuracy of each model.
    Input: dictionary with column names as keys and a tuple with reshaped arrays and CNN models
    Output: a dictionary with column names as keys and the accuracy obtained by the CNN model for that column
    """
    accuracies_dic = {}
    es = EarlyStopping(monitor='val_loss', patience=1)
    for column, tup in arrays_models_dic.items():
        print(f"Analysing {column}")
        tup[4].fit(tup[0], tup[2], epochs=10, batch_size=128, verbose=1, validation_split = 0.2, callbacks = [es])
        loss, acc = tup[4].evaluate(tup[1], tup[3], verbose=1)
        accuracies_dic[column] = f'{acc:.3f}'
    return accuracies_dic

## Selection of the best fingerprints

Select the columns containing fingerprints

In [28]:
fp_list = list(drugs.columns[5:])
fp_list

['Morgan2FP',
 'MACCSKeys',
 'AtomPairFP',
 'TopTorFP',
 'AvalonFP',
 'PubchemFP',
 'CactvsFP']

Run the functions to obtain the dictionary with the accuracies

In [29]:
# Note for the evaluator: This step may take long,
# you may want to run it partially and see that it works and then skip it and see the result below
# or decrease substantially the number of epochs
splits_dic = select_fp(fp_list)
array_model_dic= reshape_and_build(splits_dic)
accuracies_dic = fit_model(array_model_dic)

Morgan2FP
Shape X_train:  (7128, 2048) <class 'numpy.ndarray'>


array([0, 0, 0, ..., 0, 0, 0])

Number of classes:  16
Reshaped X_train:  (7128, 2048, 1)
In_shape:  (2048, 1)
Shape y_train:  (7128,)
MACCSKeys
Shape X_train:  (7128, 167) <class 'numpy.ndarray'>


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0])

Number of classes:  16
Reshaped X_train:  (7128, 167, 1)
In_shape:  (167, 1)
Shape y_train:  (7128,)
AtomPairFP
Shape X_train:  (7128, 2048) <class 'numpy.ndarray'>


...

Number of classes:  16
Reshaped X_train:  (7128, 2048, 1)
In_shape:  (2048, 1)
Shape y_train:  (7128,)
PubchemFP
Shape X_train:  (7128, 898) <class 'numpy.ndarray'>


array([1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,

Number of classes:  16
Reshaped X_train:  (7128, 898, 1)
In_shape:  (898, 1)
Shape y_train:  (7128,)
CactvsFP
Shape X_train:  (7128, 898) <class 'numpy.ndarray'>


array([1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,

Number of classes:  16
Reshaped X_train:  (7128, 898, 1)
In_shape:  (898, 1)
Shape y_train:  (7128,)
Analysing Morgan2FP
Epoch 1/10
45/45 [==============================] - 10s 207ms/step - loss: 3.2051 - accuracy: 0.2083 - val_loss: 2.1261 - val_accuracy: 0.3268
Epoch 2/10
45/45 [==============================] - 9s 201ms/step - loss: 2.1663 - accuracy: 0.3073 - val_loss: 1.8875 - val_accuracy: 0.4215
Epoch 3/10
45/45 [==============================] - 9s 201ms/step - loss: 1.9307 - accuracy: 0.3681 - val_loss: 1.7375 - val_accuracy: 0.4867
Epoch 4/10
45/45 [==============================] - 9s 204ms/step - loss: 1.7832 - accuracy: 0.4130 - val_loss: 1.6756 - val_accuracy: 0.5098
Epoch 5/10
45/45 [==============================] - 9s 206ms/step - loss: 1.7217 - accuracy: 0.4255 - val_loss: 1.6568 - val_accuracy: 0.5238
Epoch 6/10
45/45 [==============================] - 9s 207ms/step - loss: 1.6378 - accuracy: 0.4432 - val_loss: 1.6216 - val_accuracy: 0.5407
Epoch 7/10
45/45 [========

In [30]:
accuracies_dic

{'Morgan2FP': '0.556',
 'MACCSKeys': '0.484',
 'AtomPairFP': '0.253',
 'TopTorFP': '0.572',
 'AvalonFP': '0.261',
 'PubchemFP': '0.353',
 'CactvsFP': '0.353'}

With 100 epochs and a patiente of 3, the best accuracy is achieved with the Morgan fingerprints. These will be used to tune and train our CNN model.